In [ ]:
import simpy
import numpy as np
from random import uniform, normalvariate, expovariate, seed, randint

from math import floor
from scipy import stats

import matplotlib.pyplot as mp
import seaborn
seaborn.set_theme(style="darkgrid")

In [ ]:
def Average(list):            # progressive average
    temp = []
    for i in range(len(list)):
        temp.append(sum(list[0:i]) / (i+1))
    return temp

def QueueL(t=np.inf):         # Queue length at time (t)
    arr = set([int(Ă[k][1][9:]) for k in range(len(Ă)) if Ă[k][0] <= t])
    par = set([int(Ĵ[k][1][9:]) for k in range(len(Ĵ)) if Ĵ[k][0] <= t])
    return len(arr.difference(par))

def Arrival(s=0, t=np.inf):   # Arrivals in [s, t] interval
    temp = []
    for k in range(len(Ă)):
        if s <= int(Ă[k][0]) <= t:
               temp.append(int(Ă[k][1][9:]))
    return print('Number of patients arrived in [%d, %d] interval is: %d persons\nTheir ID:'%(s, t, len(temp)), temp)

def Partir(s=0, t=np.inf):    # Departures in [s, t] interval
    par = set([int(Ĵ[k][1][9:]) for k in range(len(Ĵ)) if s <= Ĵ[k][0] <= t])
    return sorted(list(par))

In [ ]:
R_Seed = 4                              # Random Seed for Psuduo random generationseed
seed(R_Seed)                            # This helps scientific proccess for checking the results
Total_Beds = 10                         # Number of Available Beds in the Hospital's ICU
λ = 10                                  # Average arrival time [in days] for a patient, ...
                                        # ... so average arrival rate is 1/λ persons per day.
CT = 7                                  # Treatment Duration in days
min_Tol = uniform(0, 1)                 # Min. patients tolerance in days
Max_Tol = uniform(1, 2)                 # Max. patients tolerance in days
First_Batch = randint(1, 10)            # Initial number of patient in day 0.0
SIM_TIME = 365                          # Simulation time in days [integer]

Ă = []                                  # Arrivals (time, name)
Ň = []                                  # Throughput (time, 0|1)
Q = []                                  # Queue legth (persons)
B = []                                  # Server utilization [Busy] (Treatment Duration = CT)
W = []                                  # Waiting time (Success wait OR Failure wait)
Ĵ = []                                  # Exact time of leaving queue (ICU or dead)

def Patient(env, name, hp):             # Patients arrives, is served if possible and leaves (dead or alive).
    arrive = env.now
    Ă.append((arrive, name))
    print('Day%7.4f: %s Arrived at Medical Center ' %(arrive, name))
    with hp.Bed.request() as request:
        patience = uniform(min_Tol, Max_Tol)
        results = yield request | env.timeout(patience)
        waitS = env.now - arrive
        if request in results:          # Patient got to the IcuBed
            W.append(waitS)
            Ĵ.append((env.now, name))
            print('Day%7.4f: %s Waited for ICU Service %5.2f days & enters the ICU on day %5.2f' 
                  %(env.now, name, waitS, env.now))
            yield env.process(hp.Cure(name))
        else:                           # Patient couldn't get to the IcuBed and failed
            waitF = env.now - arrive
            Ĵ.append((env.now, name))
            Ň.append((env.now, 0))
            W.append(waitF)
            print('Day%7.4f: %s Passed away after %7.4f days' %(env.now, name, waitF))

class Hospital(object):
    def __init__(self, env, Beds, CureTime):
        self.env = env
        self.Bed = simpy.Resource(env, Beds)
        self.CureTime = CureTime

    def Cure(self, name):
        Bedridden = normalvariate(CT, CT/6)
        yield self.env.timeout(Bedridden)
        B.append(CT)
        Ň.append((env.now, 1))
        print("Day%7.4f: %s recived medical care for %7.4f days and discharged" 
              %(env.now, name, Bedridden))

def setup(env, beds, CureTime, landa):  # Create a Hospital, a number of initial patients and go on
    hp = Hospital(env, beds, CureTime)  # Creating the Hospital
    for i in range(First_Batch):        # Create initial patients:
        env.process(Patient(env, 'Patient #%d' %(i+1), hp))
    while True:                         # Keep creating more patients every ~λ days while the simulation is running
        yield env.timeout(expovariate(landa))
        i += 1
        env.process(Patient(env, 'Patient #%d' %(i+1), hp))

print('CoViD-19 Hospital', '\n')
env = simpy.Environment()
env.process(setup(env, Total_Beds, CT, λ))
env.run(until=SIM_TIME)

In [ ]:
print ('Last patient is:', Ă[-1][1])

# Arrivals:

In [ ]:
Arrival(364, 365)

In [ ]:
A_ = [] 
for a in Ă:
    A_.append(floor(a[0]))

mp.figure(figsize=(17,4))
a1, a2, a3 = mp.hist(A_, int(SIM_TIME))
mp.plot(range(SIM_TIME), np.ones(SIM_TIME)*np.average(a1), '--')
mp.show()

In [ ]:
A_[-11:]

In [ ]:
print('Average of Arrivals: Ā = %.2f patients per day' %np.average(a1))

In [ ]:
List = a1
X = range(len(List))
Y = List
mp.figure(figsize=(6,4))
Δ = max(List) - min(List)
μ, σ = stats.norm.fit(Y)
mp.hist(Y, 17)
mp.show()
print("X ~ N(%.2f, %.2f)" %(μ, σ))
mp.figure(figsize=(17,4))
mp.title("Arrivals")
mp.plot(X, Y, label='On Each Day')
mp.plot(X, Average(Y), label='On Average')
mp.plot(X, np.ones(len(List))*np.average(Y), '--', label = 'Average Value')
mp.legend();

# Throughput 
(Proccessed units per unit of time)

In [ ]:
N = []
for k in range(len(Ň)):
    N.append(Ň[k][1])

print('Total: %.2f Persons on each day\nSuaccess: %.2f  Persons on each day\nFailure: %.2f Persons on each day' 
      %((N.count(1) + N.count(0)) / SIM_TIME, N.count(1) / SIM_TIME, N.count(0) / SIM_TIME))

In [ ]:
N_ = sum(N) / len(Ĵ)
print('Over All\nSuccess Rate: N̄ = %.2f%%' %(100*N_))

In [ ]:
List = N
X = range(len(List))
Y = List
mp.figure(figsize=(6,4))
N1, N2, N3 = mp.hist(List, 2)
print("Succes= %.02f%%\nFailure= %.2f%%" %(100*N1[1] / (N1[0]+N1[1]), 100*N1[0] / (N1[0]+N1[1]) ) )
mp.show()

mp.figure(figsize=(17,4))
mp.title("Throughput")
# mp.plot(range(len(List)), List, '.', label='On Each Day')
mp.plot(X, Average(Y), label='On Average')
mp.plot(X, np.ones(len(X))*np.average(Y), '--', label = 'Average Value')
mp.legend();

# Jobs Done

In [ ]:
print('Total jobs Done: %d personns (dead or alive)' %len(Ĵ))

In [ ]:
print('Unfinished Businesses: %d jobs' %(len(Ă) - len(Ĵ)))

In [ ]:
J = []
X = range(SIM_TIME)
for k in range(len(Ĵ)):
    J.append(floor(Ĵ[k][0]))

mp.figure(figsize=(17,4))    
j1, j2, j3 = mp.hist(J, int(SIM_TIME))
J_ = np.average(j1)

In [ ]:
print('Average of Jobs done: J̄ = %.2f patients per day' %J_)

In [ ]:
List = j1
X = range(len(List))
Y = List

mp.figure(figsize=(6,4))
Δ = int(max(List) - min(List))
μ, σ = stats.norm.fit(List)
mp.hist(List, Δ)
mp.show()
print("Jobs Done on each day ~ N(%.2f, %.2f)" %(μ, σ))

mp.figure(figsize=(17,4))
mp.title("Jobs Done")
mp.plot(range(len(List)), List, label='On Each Day')
mp.plot(range(len(List)), Average(List), label='On Average')
mp.plot(X, np.ones(len(List))*np.average(Y), '--', label = 'Average Value')
mp.legend();

# Queue Length

In [ ]:
QL = [QueueL(d) for d in range(SIM_TIME)]
Q_ = np.average(QL)
print('Average queue length: Q̄ = %.1f persons' %Q_)

In [ ]:
List = QL
X = range(len(List))
Y = List

mp.figure(figsize=(6,4))
Δ = max(List) - min(List)
μ, σ = stats.norm.fit(List)
mp.hist(List, Δ)
mp.show()
print("X ~ N(%.2f, %.2f)" %(μ, σ))

mp.figure(figsize=(17,4))
mp.title("Queue Length")
mp.plot(range(len(List)), List, label='On Each Day')
mp.plot(range(len(List)), Average(List), label='On Average')
mp.plot(X, np.ones(len(List))*np.average(Y), '--', label = 'Average Value')
mp.legend();

# Server Utilization

In [ ]:
B_ = sum(B) / (Total_Beds*SIM_TIME)
print('Average server utilization: B̄ = %.2f%%' %(100*B_))

# Waiting Time

In [ ]:
W_ = sum(W) / len(W)
print('Average time spent by individuals in queue: W̄ = %5.2f days' %W_)

In [ ]:
print ('Last 10 patients departed system (dead or alive): ', Partir(0, 365)[-10:])